In [5]:
import requests
import pandas as pd

API_TOKEN = '67fb3d6f50c489.92544905'
BASE_URL = 'https://eodhd.com/api/mp/unicornbay/options/eod'
UNDERLYING_SYMBOL = 'SOXL'
EXPIRATION_DATE = '2025-04-17'
LIMIT = 500
MAX_PAGES = 10  # Safeguard to prevent endless loops

def fetch_eod_data(strike_from, strike_to, contract_type):
    all_records = []
    page = 1
    total_expected = None  # If meta info returns total expected records
    while True:
        print(f"Fetching page {page} for {contract_type.upper()} with strikes {strike_from} to {strike_to}...")
        params = {
            'api_token': API_TOKEN,
            'filter[underlying_symbol]': UNDERLYING_SYMBOL,
            'filter[exp_date_eq]': EXPIRATION_DATE,
            'filter[strike_from]': strike_from,
            'filter[strike_to]': strike_to,
            'filter[type]': contract_type,
            'page[limit]': LIMIT,
            'page[number]': page,
            'sort': '-exp_date',  # Sort descending
            'compact': '0'
        }
        resp = requests.get(BASE_URL, params=params)
        resp.raise_for_status()
        data = resp.json()

        # Check for metadata to see total expected records (if available)
        if not total_expected and 'meta' in data and 'total' in data['meta']:
            total_expected = data['meta']['total']
            print(f"Total expected records (from meta): {total_expected}")

        if not data.get('data'):
            print(f"No more data returned on page {page}.")
            break

        page_records = data['data']
        all_records.extend(page_records)
        print(f"Page {page} complete. Retrieved {len(page_records)} records. Total collected so far: {len(all_records)}", end='')
        # If total_expected is known, print percent complete:
        if total_expected:
            percent = (len(all_records) / total_expected) * 100
            print(f" ({percent:.1f}% of expected)")
        else:
            print()

        page += 1
        if page > MAX_PAGES:
            print(f"Maximum page limit ({MAX_PAGES}) reached, stopping pagination for {contract_type.upper()} with strikes {strike_from}-{strike_to}.")
            break

    # Convert raw record list into a cleaned list with just attributes and added info.
    records_cleaned = []
    for record in all_records:
        attributes = record.get('attributes', {})
        record_id = record.get('id', '')
        attributes['record_id'] = record_id
        attributes['record_type'] = record.get('type')
        records_cleaned.append(attributes)

    print(f"Completed fetching data for {contract_type.upper()} with strikes {strike_from}-{strike_to}. Total records: {len(records_cleaned)}\n")
    return records_cleaned

# Fetch Puts
print("Starting data fetch for PUT options...")
put_data = fetch_eod_data(8.5, 13.5, 'put')

# Fetch Calls
print("Starting data fetch for CALL options...")
call_data = fetch_eod_data(8.5, 13.5, 'call')

# Combine both data sets
all_data = put_data + call_data
df = pd.DataFrame(all_data)
df.to_csv("all_strikes_8.5_to_13.5.csv", index=False)
print(f"Total records retrieved: {len(df)}")


Starting data fetch for PUT options...
Fetching page 1 for PUT with strikes 8.5 to 13.5...
Total expected records (from meta): 205
Page 1 complete. Retrieved 205 records. Total collected so far: 205 (100.0% of expected)
Fetching page 2 for PUT with strikes 8.5 to 13.5...
Page 2 complete. Retrieved 205 records. Total collected so far: 410 (200.0% of expected)
Fetching page 3 for PUT with strikes 8.5 to 13.5...
Page 3 complete. Retrieved 205 records. Total collected so far: 615 (300.0% of expected)
Fetching page 4 for PUT with strikes 8.5 to 13.5...
Page 4 complete. Retrieved 205 records. Total collected so far: 820 (400.0% of expected)
Fetching page 5 for PUT with strikes 8.5 to 13.5...
Page 5 complete. Retrieved 205 records. Total collected so far: 1025 (500.0% of expected)
Fetching page 6 for PUT with strikes 8.5 to 13.5...
Page 6 complete. Retrieved 205 records. Total collected so far: 1230 (600.0% of expected)
Fetching page 7 for PUT with strikes 8.5 to 13.5...
Page 7 complete. Retr

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4100 entries, 0 to 4099
Data columns (total 45 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   contract                 4100 non-null   object 
 1   underlying_symbol        4100 non-null   object 
 2   exp_date                 4100 non-null   object 
 3   expiration_type          4100 non-null   object 
 4   type                     4100 non-null   object 
 5   strike                   4100 non-null   float64
 6   exchange                 4100 non-null   object 
 7   currency                 4100 non-null   object 
 8   open                     4100 non-null   float64
 9   high                     4100 non-null   float64
 10  low                      4100 non-null   float64
 11  last                     4100 non-null   float64
 12  last_size                4100 non-null   int64  
 13  change                   4100 non-null   float64
 14  pctchange               

In [15]:
# Drop specified columns to create a clean dataframe
columns_to_drop = [
    'contract',
    'underlying_symbol',
    'exp_date',
    'expiration_type',
    'exchange',
    'currency',
    'bid_date',
    'ask_date',
    'tradetime',
    'record_id',
    'record_type'
]

# Create a new clean dataframe by dropping the specified columns
df_clean = df.drop(columns=columns_to_drop, errors='ignore')

# Display information about the cleaned dataframe
print("Cleaned DataFrame Info:")
df_clean.info()

# Save the cleaned dataframe to a new CSV file
print(f"Cleaned dataframe saved with {len(df_clean)} rows and {len(df_clean.columns)} columns")


Cleaned DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4100 entries, 0 to 4099
Data columns (total 34 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   type                     4100 non-null   object 
 1   strike                   4100 non-null   float64
 2   open                     4100 non-null   float64
 3   high                     4100 non-null   float64
 4   low                      4100 non-null   float64
 5   last                     4100 non-null   float64
 6   last_size                4100 non-null   int64  
 7   change                   4100 non-null   float64
 8   pctchange                4100 non-null   float64
 9   previous                 4100 non-null   float64
 10  previous_date            3000 non-null   object 
 11  bid                      4100 non-null   float64
 12  bid_size                 4100 non-null   int64  
 13  ask                      4100 non-null   float64
 14  

In [16]:
# Transform 'previous_date' to 'date' and convert to datetime
print("Converting previous_date to date and transforming to datetime format...")

# First, rename the column
df_clean = df_clean.rename(columns={'previous_date': 'date'})

# Convert the 'date' column to datetime format
df_clean['date'] = pd.to_datetime(df_clean['date'])

# Display the first few rows to verify the transformation
print("\nFirst few rows after transformation:")
print(df_clean[['date', 'type', 'strike']].head())

# Display data types to confirm conversion
print("\nData types after conversion:")
print(df_clean.dtypes[['date']])


Converting previous_date to date and transforming to datetime format...

First few rows after transformation:
        date type  strike
0 2025-04-10  put    13.5
1 2025-04-10  put    13.0
2 2025-04-10  put    12.5
3 2025-04-10  put    12.0
4 2025-04-10  put    11.5

Data types after conversion:
date    datetime64[ns]
dtype: object


In [17]:
df_clean.to_csv("all_strikes_8.5_to_13.5_clean.csv", index=False)